In [1]:
import numpy as np
import pandas as pd
import networkx as nx
import sys

input dataset: http://snap.stanford.edu/data/soc-sign-bitcoin-otc.html

In [2]:
df = np.array(pd.read_excel("datafile_bitcoin.xlsx", sheet_name = "updated_edges"))

In [3]:
edge_list = df[:,:3]
edge_list

array([[   6,    2,    4],
       [   6,    5,    2],
       [   1,   15,    1],
       ...,
       [  41, 1052,    1],
       [  13, 1128,    1],
       [1128,   13,    2]], dtype=int64)

In [4]:
node_values = np.array(pd.read_excel("datafile_bitcoin.xlsx", sheet_name = 1))

In [5]:
node_values_upd = [i for i in node_values if i<=2000]

In [6]:
g = nx.DiGraph()

In [7]:
#adding nodes
for j in node_values_upd:
    g.add_node(int(j))

In [8]:
g.number_of_nodes()

1954

In [9]:
for i in df:
    source, dest, weight ,_ = i
    g.add_edge(source,dest)
    g.add_edge(dest,source)
    g[dest][source]["w"]=0
    g[source][dest]["w"]=0

In [10]:
#adding edges and assigning weights
count = 0
for i in df:
    source, dest, weight, _ = i
    if weight >0:
        g.add_edge(source,dest)
        g[source][dest]["w"]= weight

In [11]:
def max_flow_new(G, source, dest):
    flow = 0
    def augmenting_path(G, source, dest):
        path = []
        seq = [(source, [source])]
        while seq:
            (node, path) = seq.pop(0)
            for neighbor, capacity in G[node].items():
                if capacity["w"] > 0 and neighbor not in path:
                    if neighbor == dest:
                        return path + [neighbor]
                    else:
                        seq.append((neighbor, path + [neighbor]))
        return None
    ls = []
    x = 0
    val =0
    for edge_count in range(g.number_of_edges()):
        path = augmenting_path(G, source, dest)
        if x>val:
            val = x
            ls = path
        if path:
            x = sys.maxsize
            for i in range(len(path)-1):
                if g[path[i]][path[i+1]]["w"]<x:
                    x = g[path[i]][path[i+1]]["w"]
            for i in range(len(path)-1):
                g[path[i]][path[i+1]]["w"]-=x
                g[path[i+1]][path[i]]["w"]+=x
            iter_flow = x
            flow = flow+ x
            

        else:
            print("Maximum flow possible from "+ str(source)+ " to "+ str(dest)+ " : " + str(flow))
            print("No more augumeted paths!")
            print(ls)
            break

In [12]:
max_flow_new(g,35,7)


Maximum flow possible from 35 to 7 : 220
No more augumeted paths!
[35, 322, 7]
